# VGG16


In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Input, Dropout
from tensorflow.keras import Model, Sequential

In [ ]:
tf.executing_eagerly()

True

In [ ]:
#Download cifar100 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode="fine")
print("Train feature shape",x_train.shape)
print("Train label shape",y_train.shape)
print("Test feature shape",x_test.shape)
print("Test label shape",y_test.shape)

x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

Train feature shape (50000, 32, 32, 3)
Train label shape (50000, 1)
Test feature shape (10000, 32, 32, 3)
Test label shape (10000, 1)


In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [ ]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    
    self.conv_11 = Conv2D(64, 3, activation='relu', padding ='same')
    self.conv_12 = Conv2D(64, 3, activation='relu', padding ='same')
    self.maxpool_1 = MaxPool2D(pool_size=(2, 2), strides=(2,2))

    self.conv_21 = Conv2D(128, 3, activation='relu', padding ='same')
    self.conv_22 = Conv2D(128, 3, activation='relu', padding ='same')
    self.maxpool_2 = MaxPool2D(pool_size=(2, 2), strides=(2,2))

    self.conv_31 = Conv2D(256, 3, activation='relu', padding ='same')
    self.conv_32 = Conv2D(256, 3, activation='relu', padding ='same')
    self.conv_33 =  Conv2D(256, 3, activation='relu', padding ='same')
    self.maxpool_3 = MaxPool2D(pool_size=(2, 2), strides=(2,2))

    self.conv_41 = Conv2D(512, 3, activation='relu', padding ='same')
    self.conv_42 = Conv2D(512, 3, activation='relu', padding ='same')
    self.conv_43 =  Conv2D(512, 3, activation='relu', padding ='same')
    self.maxpool_4 = MaxPool2D(pool_size=(2, 2), strides=(2,2))

    self.conv_51 = Conv2D(512, 3, activation='relu', padding ='same')
    self.conv_52 = Conv2D(512, 3, activation='relu', padding ='same')
    self.conv_53 =  Conv2D(512, 3, activation='relu', padding ='same')
    self.maxpool_5 = MaxPool2D(pool_size=(2, 2), strides=(2,2))
    self.flatten = Flatten()

    self.fc1 = Dense(4096, activation='relu')
    self.dropout1 = Dropout(rate=0.5)
    self.fc2 = Dense(4096, activation='relu')
    self.dropout2 = Dropout(rate=0.5)
    self.fc3 = Dense(1000, activation='relu')

  def call(self, x):
    #block 1
    x = self.conv_11(x)
    x = self.conv_12(x)
    x = self.maxpool_1(x)

    #block 2
    x = self.conv_21(x)
    x = self.conv_22(x)
    x = self.maxpool_2(x)

    #block 3
    x = self.conv_31(x)
    x = self.conv_32(x)
    x = self.conv_33(x)
    x = self.maxpool_3(x)

    #block 4
    x = self.conv_41(x)
    x = self.conv_42(x)
    x = self.conv_43(x)
    x = self.maxpool_4(x)

    #block 5
    x = self.conv_51(x)
    x = self.conv_52(x)
    x = self.conv_53(x)
    x = self.maxpool_5(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.dropout1(x)
    x = self.fc2(x)
    x = self.dropout2(x)
    x = self.fc3(x)
    return x

# Create an instance of the model
model = MyModel()

In [ ]:
x = Input(shape=(32,32,3), batch_size=32)
model(x)
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (32, 32, 32, 64)          1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (32, 32, 32, 64)          36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (32, 16, 16, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (32, 16, 16, 128)         73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (32, 16, 16, 128)         147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (32, 8, 8, 128)           0         
_________________________________________________________________
conv2d_4 (Conv2D)            (32, 8, 8, 256)           295

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [ ]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 2

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))

Epoch 1, Loss: 6.362151145935059, Accuracy: 1.003999948501587, Test Loss: 6.32652473449707, Test Accuracy: 1.0
Epoch 2, Loss: 6.31890869140625, Accuracy: 0.9779999852180481, Test Loss: 6.318000793457031, Test Accuracy: 1.0
